<a href="https://colab.research.google.com/github/TaniaHafsa/DIP-lab_task_1/blob/main/Cyber_bullying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import gensim

from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Flatten, Dropout, Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, accuracy_score
from gensim.models import FastText
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# importing the Dataset

import pandas as pd

messages = pd.read_csv('/content/drive/MyDrive/text dataset/text.txt', sep='\t',
                           names=["message", "label"])

In [ ]:
messages

,message,label
0,[ 1. 0. 0. 0. 0. 0. 0.] During the perio...,NaN
1,[ 0. 1. 0. 0. 0. 0. 0.] When I was invol...,NaN
2,[ 0. 0. 1. 0. 0. 0. 0.] When I was drivi...,NaN
3,[ 0. 0. 0. 1. 0. 0. 0.] When I lost the ...,NaN
4,[ 0. 0. 0. 0. 1. 0. 0.] The time I knock...,NaN
...,...,...
7475,[ 0. 0. 1. 0. 0. 0. 0.] Two years back s...,NaN
7476,[ 0. 0. 0. 1. 0. 0. 0.] I had taken the ...,NaN
7477,[ 0. 0. 0. 0. 1. 0. 0.] I was at home an...,NaN
7478,[ 0. 0. 0. 0. 0. 1. 0.] I did not do the...,NaN


In [ ]:
messages['message'].loc[451]

'[ 0.  0.  0.  0.  0.  1.  0.] When I was young, about 6 or 7, I did not make it to the toilet on time.  I hid the dirty trousers in the cellar as I feared itbeing discovered by my parents.'

In [ ]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['period fall love time met especi met long time',
 'involv traffic accid',
 'drive home sever day hard work motorist ahead drive km hour refus despit low speeed let overtak',
 'lost person meant',
 'time knock deer sight anim injuri helpless realiz anim badli hurt put anim scream moment death',
 'speak truth',
 'caus problem somebodi could keep appoint time led variou consequ',
 'got letter offer summer job appli',
 'go home alon one night pari man came behind ask afraid alon late night',
 'talk parti first time long friend came interrupt us left',
 'friend ask go new year parti',
 'saw drunk kid year old town walpurgi night',
 'could rememb say present task account meet',
 'uncl neighbour came home polic escort',
 'day feel close partner friend feel peac also experi close contact peopl regard greatli',
 'everi time imagin someon love could contact seriou ill even death',
 'obvious unjustli treat possibl elucid',
 'think short time live relat period life think use short time',
 'gathe

In [ ]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500,binary=True,ngram_range=(2,2))
X = cv.fit_transform(corpus).toarray()

In [ ]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
X.shape

(7480, 2500)

In [ ]:
y=pd.get_dummies(messages['message'])
y=y.iloc[:,1].values

In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [ ]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
score=accuracy_score(y_test,y_pred)
print(score)

1.0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1496

    accuracy                           1.00      1496
   macro avg       1.00      1.00      1.00      1496
weighted avg       1.00      1.00      1.00      1496



In [ ]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X = tv.fit_transform(corpus).toarray()

In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [ ]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [ ]:
score=accuracy_score(y_test,y_pred)
print(score)

1.0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1496

    accuracy                           1.00      1496
   macro avg       1.00      1.00      1.00      1496
weighted avg       1.00      1.00      1.00      1496



In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred=classifier.predict(X_test)

In [ ]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1496

    accuracy                           1.00      1496
   macro avg       1.00      1.00      1.00      1496
weighted avg       1.00      1.00      1.00      1496



# Word2vec Implementation

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim.downloader as api

In [ ]:
DIM = 100

X = corpus
w2v_model = gensim.models.FastText(sentences = X, vector_size = DIM, window = 10, min_count = 1, sg = 1)

In [ ]:
len(w2v_model.wv.key_to_index.keys())

27

In [ ]:
w2v_model.wv.most_similar('friend')

[('j', 0.23905526101589203),
 (' ', 0.13379758596420288),
 ('q', 0.12984499335289001),
 ('s', 0.127687469124794),
 ('z', 0.1129714772105217),
 ('i', 0.10629113763570786),
 ('d', 0.10297034680843353),
 ('u', 0.09652853012084961),
 ('b', 0.08634782582521439),
 ('n', 0.07863657176494598)]

In [ ]:
w2v_model.wv.most_similar('child')

[('o', 0.24283532798290253),
 ('u', 0.2169032096862793),
 ('g', 0.21497058868408203),
 ('v', 0.21127520501613617),
 ('j', 0.1927260011434555),
 ('d', 0.18862126767635345),
 ('t', 0.17823031544685364),
 ('r', 0.16509994864463806),
 ('q', 0.13428400456905365),
 ('y', 0.12750127911567688)]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['period falling love time met especially met long time',
 'involved traffic accident',
 'driving home several day hard work motorist ahead driving km hour refused despite low speeed let overtake',
 'lost person meant',
 'time knocked deer sight animal injury helplessness realization animal badly hurt put animal screamed moment death',
 'speak truth',
 'caused problem somebody could keep appointed time led various consequence',
 'got letter offering summer job applied',
 'going home alone one night paris man came behind asked afraid alone late night',
 'talking party first time long friend came interrupted u left',
 'friend ask go new year party',
 'saw drunk kid year old town walpurgis night',
 'could remember say presentation task account meeting',
 'uncle neighbour came home police escort',
 'day feel close partner friend feel peace also experience close contact people regard greatly',
 'every time imagine someone love could contact serious illness even death',
 'obviously unjustly 

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
corpus[0]

'period falling love time met especially met long time'

In [ ]:
simple_preprocess

<function gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)>

In [ ]:
import nltk
nltk.download('punkt')
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
words

[['period',
  'falling',
  'love',
  'time',
  'met',
  'especially',
  'met',
  'long',
  'time'],
 ['involved', 'traffic', 'accident'],
 ['driving',
  'home',
  'several',
  'day',
  'hard',
  'work',
  'motorist',
  'ahead',
  'driving',
  'km',
  'hour',
  'refused',
  'despite',
  'low',
  'speeed',
  'let',
  'overtake'],
 ['lost', 'person', 'meant'],
 ['time',
  'knocked',
  'deer',
  'sight',
  'animal',
  'injury',
  'helplessness',
  'realization',
  'animal',
  'badly',
  'hurt',
  'put',
  'animal',
  'screamed',
  'moment',
  'death'],
 ['speak', 'truth'],
 ['caused',
  'problem',
  'somebody',
  'could',
  'keep',
  'appointed',
  'time',
  'led',
  'various',
  'consequence'],
 ['got', 'letter', 'offering', 'summer', 'job', 'applied'],
 ['going',
  'home',
  'alone',
  'one',
  'night',
  'paris',
  'man',
  'came',
  'behind',
  'asked',
  'afraid',
  'alone',
  'late',
  'night'],
 ['talking',
  'party',
  'first',
  'time',
  'long',
  'friend',
  'came',
  'interrupt

In [ ]:
import gensim

In [ ]:
#lets train word2vec fromscratch
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [ ]:
model.wv.index_to_key

['friend',
 'felt',
 'one',
 'time',
 'day',
 'got',
 'year',
 'mother',
 'home',
 'would',
 'told',
 'school',
 'people',
 'person',
 'went',
 'girl',
 'saw',
 'father',
 'first',
 'could',
 'night',
 'came',
 'go',
 'made',
 'mine',
 'boyfriend',
 'exam',
 'car',
 'back',
 'someone',
 'something',
 'found',
 'parent',
 'angry',
 'sister',
 'brother',
 'thought',
 'good',
 'close',
 'died',
 'wanted',
 'man',
 'another',
 'see',
 'away',
 'feel',
 'asked',
 'get',
 'class',
 'know',
 'work',
 'going',
 'thing',
 'girlfriend',
 'child',
 'passed',
 'guilty',
 'feeling',
 'two',
 'way',
 'ashamed',
 'house',
 'like',
 'said',
 'sad',
 'without',
 'heard',
 'much',
 'left',
 'alone',
 'boy',
 'university',
 'old',
 'later',
 'last',
 'really',
 'family',
 'long',
 'started',
 'well',
 'room',
 'place',
 'teacher',
 'disgusted',
 'money',
 'result',
 'failed',
 'examination',
 'love',
 'afraid',
 'bad',
 'student',
 'week',
 'took',
 'take',
 'tried',
 'come',
 'life',
 'met',
 'late',
 '

In [ ]:
model.corpus_count

7480

In [ ]:
model.epochs

5

In [ ]:
model.wv.similar_by_word('child')

[('took', 0.999786376953125),
 ('saw', 0.9997787475585938),
 ('sister', 0.9997687935829163),
 ('life', 0.9997685551643372),
 ('woman', 0.9997649788856506),
 ('put', 0.9997603893280029),
 ('boy', 0.9997564554214478),
 ('people', 0.999754011631012),
 ('brother', 0.9997532367706299),
 ('started', 0.9997525215148926)]

In [ ]:
model.wv['kid'].shape

(100,)

In [ ]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
words[75]

['told',
 'friend',
 'going',
 'party',
 'would',
 'enjoy',
 'came',
 'along',
 'understood',
 'would',
 'fitted',
 'something']

In [ ]:
type(model.wv.index_to_key)

list

In [ ]:
words

[['period',
  'falling',
  'love',
  'time',
  'met',
  'especially',
  'met',
  'long',
  'time'],
 ['involved', 'traffic', 'accident'],
 ['driving',
  'home',
  'several',
  'day',
  'hard',
  'work',
  'motorist',
  'ahead',
  'driving',
  'km',
  'hour',
  'refused',
  'despite',
  'low',
  'speeed',
  'let',
  'overtake'],
 ['lost', 'person', 'meant'],
 ['time',
  'knocked',
  'deer',
  'sight',
  'animal',
  'injury',
  'helplessness',
  'realization',
  'animal',
  'badly',
  'hurt',
  'put',
  'animal',
  'screamed',
  'moment',
  'death'],
 ['speak', 'truth'],
 ['caused',
  'problem',
  'somebody',
  'could',
  'keep',
  'appointed',
  'time',
  'led',
  'various',
  'consequence'],
 ['got', 'letter', 'offering', 'summer', 'job', 'applied'],
 ['going',
  'home',
  'alone',
  'one',
  'night',
  'paris',
  'man',
  'came',
  'behind',
  'asked',
  'afraid',
  'alone',
  'late',
  'night'],
 ['talking',
  'party',
  'first',
  'time',
  'long',
  'friend',
  'came',
  'interrupt

In [ ]:
import numpy as np

In [ ]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

 15%|█▍        | 1104/7480 [00:00<00:01, 5031.66it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 7480/7480 [00:01<00:00, 5657.03it/s]


In [ ]:
type(X)

list

In [ ]:
X_new=np.array(X)

<ipython-input-69-1dbb21e0886e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_new=np.array(X)


In [ ]:
X_new[0]

array([-0.2310384 ,  0.2033418 ,  0.01577657,  0.0862587 ,  0.01437774,
       -0.37179047,  0.3057525 ,  0.7626595 , -0.20332968,  0.03277812,
       -0.019258  , -0.53097034, -0.06980441,  0.13230838,  0.09410948,
       -0.33906868,  0.16858956, -0.46892172, -0.16696039, -0.70729476,
        0.0710758 ,  0.14501968, -0.01564419, -0.09156109, -0.06977735,
       -0.13027337, -0.07056879, -0.47333324, -0.41328225, -0.089421  ,
        0.25615466,  0.07479858,  0.15489212, -0.08242967, -0.06538034,
        0.3840769 ,  0.12512574, -0.37560076, -0.31839693, -0.97016716,
        0.01585419, -0.369249  , -0.13245822,  0.03459953,  0.13421983,
       -0.23856789, -0.48325765, -0.09884354,  0.23000023,  0.338211  ,
        0.07499178, -0.39851362, -0.23392685,  0.06406519, -0.13907644,
        0.24530858,  0.30697945,  0.02944501, -0.5193143 ,  0.15029973,
        0.0805469 ,  0.06943095, -0.24362892, -0.10128462, -0.3977338 ,
        0.29148337,  0.14871642,  0.3329752 , -0.71188045,  0.41

In [ ]:
X_new[0].shape

(100,)

In [ ]:
X_new[3]

array([-0.25598422,  0.23119068,  0.01616619,  0.09637239,  0.00784549,
       -0.41524616,  0.3390812 ,  0.8483024 , -0.23316711,  0.04165715,
       -0.03651403, -0.6071573 , -0.09042902,  0.14535576,  0.10155866,
       -0.3776176 ,  0.18298157, -0.52552223, -0.18527246, -0.8040449 ,
        0.07838982,  0.17172836, -0.01826259, -0.09775021, -0.07869827,
       -0.15063275, -0.09897039, -0.53876656, -0.47860122, -0.10468508,
        0.2856233 ,  0.09412795,  0.17654197, -0.09233766, -0.06567666,
        0.43457744,  0.12175534, -0.41967848, -0.3612117 , -1.0956662 ,
        0.02135357, -0.41203117, -0.16582294,  0.04673626,  0.1448612 ,
       -0.28007978, -0.5496176 , -0.11017071,  0.26576045,  0.39044642,
        0.09253842, -0.44682357, -0.26328424,  0.0775393 , -0.1539017 ,
        0.2853537 ,  0.35011896,  0.0341691 , -0.5920956 ,  0.18339849,
        0.10221206,  0.07746211, -0.2809018 , -0.11219829, -0.4541067 ,
        0.3274717 ,  0.16999952,  0.3795829 , -0.80156946,  0.46

In [ ]:
X_new.shape

(7480,)